## Transfer Resting State Eyetracking Files to USB from Host Computer (.ps1)

VERSION: `Structural_Eyetrack_Auto_Pull`

### Advanced Explanation

The script is designed to selectively transfer directories from a source to a destination path based on user input. It first defines the source (`$source`) and destination (`$destination`) paths. Using `Get-ChildItem`, it retrieves subdirectories in `$source` that were last written to within the last 10 hours.

In the loop over each directory (`$dir`) in `$rawdata`, the script calculates the size in megabytes of each directory by summing the size of all files within (`Get-ChildItem -Recurse -File` with `Measure-Object`). It then prompts the user with details of the directory (name, size, creation time), asking if they wish to transfer it to the destination (`$destination`). If the user inputs "Y", the script uses `Copy-Item -Recurse` to copy the entire directory to the destination, confirming each copy with an output message.

### User-Friendly Explanation

This script is a tool for backing up folders from one part of your computer to a USB drive. It looks at specific folders that have been updated in the last 10 hours. For each of these folders, it shows you the folder's name, size, and when it was created, and asks if you want to back it up to your USB.

If you choose to back up a folder, it copies the entire folder to the USB. The script tells you once it has successfully copied a folder. This way, you can selectively back up recent folders to your USB, ensuring you have the latest data saved.

### Oversimplified Explantion

This script will treat session folders created within 10 hours as new data and copy it over to the USB. A prompt will appear asking for confirmation of the files that you would like to collect.  

In [ ]:
$source = "C:\Users\at-risk\Documents\atrisk_nate\ARD2_RestingState_Deployed\results"
$destination = "E:\RestingState_DAY1"
$currentDate = Get-Date
$rawdata = Get-ChildItem -Path $source -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

foreach ($dir in $rawdata) {
    $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
    $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
    $creationTime = $dir.CreationTime
    $userInput = Read-Host "Do you want to pull $($dir.Name) [Size: $($dirSizeMB) MB, Created: $($creationTime)]
     from $source to the USB? [Y/N]"

    if ($userInput -eq "Y") {
        $destPath = Join-Path -Path $destination -ChildPath $dir.Name
        Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
        Write-Output "Copied $($dir.Name) to USB"
    }
}